In [1]:
# %load ir_lmart.py
#!/usr/bin/env python

# In[1]:


# Stronger baseline: Listwise L2R - LambdaMART
# Hyperparameter optimziation HPonsteroids requires Python 3!


# In[2]:


# Imports
import os
import subprocess
import sys
from functools import partial
import multiprocessing
import numpy as np
import pickle
import json

# REMOVE!!
from eval_utils import *

# HPO
from hpo_utils import *

from ir_utils import *

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS
from hpbandster.examples.commons import MyWorker

In [2]:
# In[3]:


# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        


# In[4]:


# Classes
class L2Ranker:
    def __init__(self, ranklib_location, params, normalization=[]):
        self.ranklib_location = ranklib_location
        # Works with Oracle JSE
        # java version "1.8.0_211"
        # Java(TM) SE Runtime Environment (build 1.8.0_211-b12)
        # Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
        self.params = params
        self.ranker_command = ['java', '-jar', ranklib_location + 'RankLib-2.12.jar']
        self.normalization = normalization
        self.save_model_file = ''
        
#     def build(self, ir_tool_params):
    def train(self, train_data_file, save_model_file, hpo_config):
        self.save_model_file = save_model_file
        self.log_file = save_model_file + '.log'
        self.hpo_config= hpo_config
        toolkit_parameters = [
                                *self.ranker_command, # * to unpack list elements
                                '-train',
                                train_data_file,
                                *self.normalization,
                                *self.params,
                                '-leaf', 
                                str(self.hpo_config['n_leaves']),
                                '-shrinkage',
                                str(self.hpo_config['learning_rate']),
                                '-tree', # Oner regression tree per boosted iteration
                                str(self.hpo_config['n_trees']),
                                '-save',
                                self.save_model_file   
                            ] 
        
        print(toolkit_parameters)
        with open(self.log_file, 'wt') as rf:
            proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=rf, stderr=subprocess.PIPE, shell=False)
#         proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
            
        (out, err)= proc.communicate()
#         print(out.decode('utf-8').splitlines())
#         print(out)

        if err == b'':
            print('Model saved: ', self.save_model_file)
        else:
#             print('error:', err, type(err))
            print('Something went wrong on training, see log: ', self.log_file)
            
  

    def gen_run_file(self, test_data_file, run_file):
        pre_run_file = run_file.replace('run_', 'pre_run_', 1)
        toolkit_parameters = [
                                *self.ranker_command, # * to unpack list elements
                                '-load',
                                self.save_model_file,
                                *self.normalization,
                                '-rank',
                                test_data_file,
                                '-indri',
                                pre_run_file     
                            ] 
        
#         print(toolkit_parameters)
        with open(self.log_file, 'at') as rf:
            proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=rf, stderr=subprocess.STDOUT, shell=False)
#         proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
            
        (out, err)= proc.communicate()
#         print(out.decode('utf-8').splitlines())
#         print(out)
        print(err)
    
        print(run_file)
        print(pre_run_file)
        
        generate_run_file(pre_run_file, run_file)
        
#         print('Run model saved: ', run_file)


# In[5]:


# try:
#     import keras
#     from keras.datasets import mnist
#     from keras.models import Sequential
#     from keras.layers import Dense, Dropout, Flatten
#     from keras.layers import Conv2D, MaxPooling2D
#     from keras import backend as K
# except:
#     raise ImportError("For this example you need to install keras.")

# try:
#     import torchvision
#     import torchvision.transforms as transforms
# except:
#     raise ImportError("For this example you need to install pytorch-vision.")

In [3]:
class fakeParser:
    def __init__(self):
        self.min_budget = 2 
        self.max_budget = 4
        self.n_iterations = 4 
        self.n_workers =4
        self.dataset = 'bioasq' 
        self.data_split = 'test'
#         self.data_split = 'train'
#         self.data_split = 'dev'
#         self.build_index = True
        self.build_index = None
        self.fold = '1'
        self.gen_features = True
#         self.gen_features = None
        
# In[6]:


# In[3]:

In [4]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=float, help='Minimum budget used during the optimization.',    default=2)
    parser.add_argument('--max_budget',   type=float, help='Maximum budget used during the optimization.',    default=4)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)
    
    parser.add_argument('--dataset',   type=str, help='')
    parser.add_argument('--data_split',   type=str, help='')
    parser.add_argument('--pool_size',   type=int, help='')
#     parser.add_argument('--build_index', action='store_true')
    parser.add_argument('--fold', type=str,   help='')
#     parser.add_argument('--gen_features', action='store_true')

#     args=parser.parse_args()
    args = fakeParser()
    
#     dataset = sys.argv[1] # 'bioasq'
#     workdir = './' + dataset + '_dir/'
#     data_split = sys.argv[2] # 'test'

    workdir = './workdir/'
    confdir = './tvqa_config/'
    gen_features_dir = workdir + 'gen_features_dir/'
    ranklib_location = '../ranklib/'
    
    
    
    train_data_file = gen_features_dir + 'l2r_features_train'
    val_data_file = gen_features_dir + 'l2r_features_dev'
    test_data_file = gen_features_dir + 'l2r_features_test'
    
    qrels_val_file = workdir + 'gold_answer_qrels_dev'

    l2r_model = 'lmart'

    enabled_features_file = confdir + 'tvqa' + '_' + l2r_model + '_enabled_features' # dont'f change to fold_dir!

#     print(enabled_features_file)
    # Train L2R model: LambdaMART
    # Parameters 

    n_leaves = '10'
    learning_rate = '0.1'
    n_trees = '1000'
    hpo_params = {'n_leaves': n_leaves, 'learning_rate': learning_rate, 'n_trees': n_trees}



    metric2t = 'P@1' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'

    ranker_type = '6' # LambdaMART

    # normalization: Feature Engineering?
#     norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'

    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'

    l2r_params = [
        '-validate',
        val_data_file,
        '-ranker',
        ranker_type,
        '-metric2t',
        metric2t,
        '-feature',
        enabled_features_file
    ]

    # Run train

    lmart_model = L2Ranker(ranklib_location, l2r_params)
#     lmart_model = L2Ranker(ranklib_location, l2r_params, norm_params)


In [5]:



params_list = [
    train_data_file,
    val_data_file,
    lmart_model,
    qrels_val_file
]

def eval_hpo(params_list, hpo_params):
    
    train_data_file = params_list[0]
    val_data_file = params_list[1]
    lmart_model = params_list[2]
    qrels_val_file = params_list[3]
    
    hpo_params_suffix = 'nl' + str(hpo_params['n_leaves']) + 'lr' + str(hpo_params['learning_rate']) + 'nt' + str(hpo_params['n_trees'])
    
    save_model_file = workdir + 'lmart_' + hpo_params_suffix + '_model' 
    
    lmart_model.train(train_data_file, save_model_file, hpo_params)
    run_file = workdir + 'run_' + '_lmart_' + hpo_params_suffix
    
#   lmart_model.gen_run_file(test_data_file, run_file)

    lmart_model.gen_run_file(val_data_file, run_file)
    
#     print(qrels_val_file)
#     print(run_file)
    eval_results = 'Nothing by now r_r'
#     eval_results.update(lmart_model.hpo_config)
#     eval_results['lmart_model'] = lmart_model
    return eval_results

In [6]:
# def find_best_dev_model(best_model_params_file, random_iterations = 5000):
# #     random_search = 'yes'
    
#     if random_search == 'yes':
#         ## Heavy random search
#         brange = np.arange(0.1,1,0.05)
#         krange = np.arange(0.1,4,0.1)
#         N_range = np.arange(5,500,1) # num of docs
#         M_range = np.arange(5,500,1) # num of terms
#         lamb_range = np.arange(0,1,0.1) # weights of original query

#         ## Light random search
# #         brange = [0.2]
# #         krange = [0.8]
# #         N_range = np.arange(1,50,2)
# #         M_range = np.arange(1,50,2)
# #         lamb_range = np.arange(0,1,0.2)
        
#         h_param_ranges = [brange, krange, N_range, M_range, lamb_range]
#         params = get_random_params(h_param_ranges, random_iterations)

#     else:
#         brange = [0.2]
#         krange = [0.8]
#         N_range = [11]
#         M_range = [10]
#         lamb_range = [0.5]
       
#         params = [[round(b,3), round(k,3), round(N,3), round(M,3), round(Lambda,3)] 
#                   for b in brange for k in krange for N in N_range for M in M_range for Lambda in lamb_range]
   
#     print('# Params: ', len(params)) 
#     pool_size = 20
# #     print(len(params))
#     pool = multiprocessing.Pool(processes=pool_size,
#                                 initializer=start_process,
#                                 )

# #     pool_outputs = pool.map(bm25_computing, params)
    

#     pool_outputs = pool.map_async(bm25_computing, params)
# #     pool_outputs.get()
#     ###

    
#     ##
    
    
#     pool.close() # no more tasks
#     while (True):
#         if (pool_outputs.ready()): break
#         remaining = pool_outputs._number_left
# #         remaining2 = remaining1
# #         remaining1 = pool_outputs._number_left
#         if remaining%10 == 0:
#             print("Waiting for", remaining, "tasks to complete...")
#             time.sleep(2)
        
      
#     pool.join()  # wrap up current tasks
#     pool_outputs.get()
#     params_file = './best_ir_model/' + dataset_name_ext + '_' + 'bm25_rm3_' + split + '_hparams.pickle'
#     pickle.dump(pool_outputs.get(), open(params_file, "wb" ) )
#     print('Total parameters: ' + str(len(pool_outputs.get())))
#     best_model_params = max(pool_outputs.get(), key=lambda x: x[5])
    
#     best_model_dict = {
#         'b': best_model_params[0],
#         'k': best_model_params[1],
#         'N': best_model_params[2],
#         'M': best_model_params[3],
#         'Lambda': best_model_params[4],
#         'random_iterations': random_iterations,
#         'map': best_model_params[5],
#         'p_20': best_model_params[6],
#         'ndcg_20': best_model_params[7]
        
#     }
#     best_model_dict = {k:str(v) for k, v in best_model_dict.items()} # everything to string
    
#     with open(best_model_params_file, 'wt') as best_model_f:
#         json.dump(best_model_dict, best_model_f)


In [7]:
def start_process():
    print( 'Starting', multiprocessing.current_process().name)


In [8]:
def eval_multi_hpo(params_list, hpo_params_list, pool_size):
   
    eval_hpo_partial = partial(eval_hpo, params_list)

    pool = multiprocessing.Pool(processes=pool_size,
                                initializer=start_process,
                                )

    pool_outputs = pool.map_async(eval_hpo_partial, hpo_params_list)
    pool.close() # no more tasks
    pool.join()  # wrap up current tasks
    print('Total parameters: ' + str(len(pool_outputs.get())))
    return pool_outputs.get()

In [9]:
hpo_method = 'rs'

random_iterations = 20 # these are outside parameters

nleaves_range = np.arange(1,51,1)
lrate_range = np.arange(0.1,1,0.1)
ntrees_range = np.arange(1,51,1)

h_param_ranges = [nleaves_range, lrate_range, ntrees_range]

if hpo_method == 'rs':
    h_params = get_random_params(h_param_ranges, random_iterations)
elif hpo_method == 'gs':
    h_params = get_grid_search_params(h_param_ranges)

hpo_params_list = [{'n_leaves': x[0], 'learning_rate': x[1], 'n_trees': x[2]} for x in h_params]

print(len(hpo_params_list))

20


In [10]:
# hpo_params_list = [hpo_params]
pool_size = 5

hpo_results = eval_multi_hpo(params_list, hpo_params_list, pool_size)

Starting ForkPoolWorker-1
Starting ForkPoolWorker-3
Starting ForkPoolWorker-2
Starting ForkPoolWorker-4
['java', '-jar', '../ranklib/RankLib-2.12.jar', '-train', './workdir/gen_features_dir/l2r_features_train', '-validate', './workdir/gen_features_dir/l2r_features_dev', '-ranker', '6', '-metric2t', 'P@1', '-feature', './tvqa_config/tvqa_lmart_enabled_features', '-leaf', '33', '-shrinkage', '0.9', '-tree', '10', '-save', './workdir/lmart_nl33lr0.9nt10_model']
['java', '-jar', '../ranklib/RankLib-2.12.jar', '-train', './workdir/gen_features_dir/l2r_features_train', '-validate', './workdir/gen_features_dir/l2r_features_dev', '-ranker', '6', '-metric2t', 'P@1', '-feature', './tvqa_config/tvqa_lmart_enabled_features', '-leaf', '47', '-shrinkage', '0.6', '-tree', '8', '-save', './workdir/lmart_nl47lr0.6nt8_model']
['java', '-jar', '../ranklib/RankLib-2.12.jar', '-train', './workdir/gen_features_dir/l2r_features_train', '-validate', './workdir/gen_features_dir/l2r_features_dev', '-ranker', '6

In [11]:
best_model_hpo = max(hpo_results, key=lambda x: x['map'])
best_model_hpo['random_iterations'] = random_iterations
best_model_hpo['data_split'] = 'validation'
best_lmart_model = best_model_hpo.pop('lmart_model')
best_model_hpo = {k: str(v) for k, v in best_model_hpo.items()}

hpo_results_file = workdir + '_' + fold + '_' + l2r_model + '_hpo_results.pickle'
best_model_hpo_file = workdir  +  'best_' + dataset + '_' + fold + '_' + l2r_model + '_hparams.json'

pickle.dump(hpo_results, open(hpo_results_file, "wb" ) )

TypeError: string indices must be integers

In [ ]:
hpo_results

In [ ]:
qrels_test_file = workdir + '_' + 'test_' + fold + '_qrels'

run_file = workdir + 'run_' + dataset + '_' + fold + '_test_lmart_best'

best_lmart_model.gen_run_file(test_data_file, run_file)

eval_results = eval(trec_eval_command, qrels_test_file, run_file)

In [ ]:
eval_results['data_split'] = 'test'
eval_results

best_model_hpo['test'] = eval_results

with open(best_model_hpo_file, 'wt') as best_model_f:
    json.dump(best_model_hpo, best_model_f)